## Create VASP input decks of Pd-H system for H-H interaction analysis
### Script for $\text{Pd}_{64}\text{H}_1 \rightarrow$ Palladium with one hydrogen

In [1]:
import numpy as np
import os
import crystaltools.crystal_generator as cg
import crystaltools.vasp_job as vj
import matplotlib.pyplot as plt
import copy
from shutil import copyfile

%matplotlib inline

In [2]:
root_dir = './Pd_H_calculations/kpt888/Pd64H1'
path_to_POTCAR = '/Users/dagatkarimlan/python/vaspScripts/potcars/POTCAR.paw-gga-pbe-v5.2.PdH'
path_to_INCAR = '/Users/dagatkarimlan/python/vaspScripts/inputFiles/INCAR_PdH'
path_to_KPOINTS = '/Users/dagatkarimlan/python/vaspScripts/inputFiles/KPOINTS_8x8x8'
path_to_support_dir = '/Users/dagatkarimlan/python/vaspScripts/supportFiles'
support_file_list = ['run.vasp5.par.cluster.cmmc_40cores', 
                     'submit_vaspjob', 'script.parallel.sh']

path_to_support_global = '/Users/dagatkarimlan/python/vaspScripts/supportFilesGlobal'
global_file_list = ['launch_jobs_rgz.py']

In [7]:
# data from Aydin's data on Pd32H1 with H on the octahedral site
nhost = 32
nH = 1
E0 = -169.285059
V0 = 496.882551
B0 = 167.686209
B0p = 5.602486
DeltaE = -3.503565
DeltaV = 2.647853
DeltaB = -0.456442
DeltaBp = -0.012335

In [8]:
# atomic volume and misfit volume
vhost = (V0-DeltaV)/nhost
dvH = DeltaV

var = (vhost*4.)**(1./3.)

print('Atomic volume host atom: {} A^3'.format(vhost))
print('Misfit volume of H in host: {} A^3'.format(dvH))
print(var)

Atomic volume host atom: 15.4448343125 A^3
Misfit volume of H in host: 2.647853 A^3
3.95319054829


In [9]:
# Define parameters of the supercell
nsc = [4, 4, 4]
nH = 1
a0 = (4.*vhost+(1./nsc[0]**3.)*nH*dvH)**(1./3.)

print('lattice paramter: {} A'.format(a0))

lattice paramter: 3.95407281568 A


In [3]:
def create_cell_with_2H(a0, nsc=[4, 4, 4], ii=0):
    """
        Creates an fcc supercell based on the primitive cell with 
        2 hydrogen atoms with specific separation distance.
        
        :params:
            a0:(float): 
                Lattice parameter of the supercell.
            nsc:(list):(default=[4, 4, 4])
                List defining the size of the supercell.
            ii:(integer):(default=1)
                (ii+1)^th nearest neighbor distances, 
                i.e., ii=0 corresponds to the 1st NN,
                ii=4 corresponds to the 5th NN.
                
        :return:
            crystal:(cg.Crystal):
                The desired crystal crystal structure.
            dH-H:(float):
                The hydrogen-hydrogen separation in the simulation
                cell.
            
    """
    
    # create supercell (rocksalt = fcc with full occupancy of octahedral sites)
    unit = cg.get_unit_cell('rocksalt primitive', a0)
    crystal = unit.create_supercell(nsc[0], nsc[1], nsc[2])
    
    # get index of hydrogen atoms
    idH = np.nonzero(crystal.itype==2)[0]
    
    # center cell on the first H atom
    rtrans = crystal.rcar[idH[0]]
    crystal.translate_cartesian(rtrans)
    crystal.center_atoms()

    # calculate neighbor distances
    rcar_H = crystal.rcar[idH]
    dist_neigh = np.linalg.norm(rcar_H, axis=1)
    dist_neigh = np.round(dist_neigh/a0, 5) * a0
    dist_unique = np.unique(dist_neigh)[1:]
    print(dist_unique/a0)
    
    if ii > len(dist_unique)-1:
        print('Index out of range.  Max index = {}'.format(len(dist_unique)))
        return

    # set min and max bounds for neighbor search
    if ii == 0:
        dmin = 0.
    else:
        dmin = (dist_unique[ii] + dist_unique[ii-1])/2.

    if ii == len(dist_unique)-1:
        dmax = a0 * nsc[0] # some large number
    else:
        dmax = (dist_unique[ii] + dist_unique[ii+1])/2.

    # get indices of sites with the right distance
    id_nspecific = np.nonzero((dist_neigh>dmin) & (dist_neigh<dmax))[0]

    # Keep the 2 relevant H, delete all other octahedral sites
    crystal.itype[idH] = 3
    crystal.itype[idH[0]] = 2
    crystal.itype[idH[id_nspecific[0]]] = 2
    iddel = np.nonzero(crystal.itype == 3)[0]
    crystal.delete_atom(iddel)
    
    # shift atoms back to original positions
    crystal.translate_cartesian(-rtrans)
    crystal.reflect_atoms()
    
    return crystal, dist_unique[ii]

In [4]:
def create_cell_with_1H(a0, nsc=[4, 4, 4]):
    """
        Creates an fcc supercell based on the primitive cell with 
        1 hydrogen atom.
        
        :params:
            a0:(float): 
                Lattice parameter of the supercell.
            nsc:(list):(default=[4, 4, 4])
                List defining the size of the supercell.
                
        :return:
            crystal:(cg.Crystal):
                The desired crystal crystal structure.
    """
    # create supercell (rocksalt = fcc with full occupancy of octahedral sites)
    unit = cg.get_unit_cell('rocksalt primitive', a0)
    crystal = unit.create_supercell(nsc[0], nsc[1], nsc[2])
    
    # get index of hydrogen atoms
    idH = np.nonzero(crystal.itype==2)[0]
    
    # Keep the 2 relevant H, delete all other octahedral sites
    crystal.itype[idH] = 3
    crystal.itype[idH[0]] = 2
    iddel = np.nonzero(crystal.itype == 3)[0]
    crystal.delete_atom(iddel)
    
    return crystal

In [5]:
def create_bulk_cell(a0, nsc=[4, 4, 4]):
    """
        Creates an fcc supercell based on the primitive cell with 
        no hydrogen atoms.
        
        :params:
            a0:(float): 
                Lattice parameter of the supercell.
            nsc:(list):(default=[4, 4, 4])
                List defining the size of the supercell.
                
        :return:
            crystal:(cg.Crystal):
                The desired crystal crystal structure.
    """
    # create supercell (rocksalt = fcc with full occupancy of octahedral sites)
    unit = cg.get_unit_cell('fcc primitive', a0)
    crystal = unit.create_supercell(nsc[0], nsc[1], nsc[2])
    
    return crystal

In [6]:
def create_rocksalt_cell(a0, nsc=[4, 4, 4]):
    """
        Creates an rocksalt supercell based on the primitive cell (i.e. fcc
        supercell with full occupancy of the octahedral sites).
        
        :params:
            a0:(float): 
                Lattice parameter of the supercell.
            nsc:(list):(default=[4, 4, 4])
                List defining the size of the supercell.
                
        :return:
            crystal:(cg.Crystal):
                The desired crystal crystal structure.
    """
    # create supercell (rocksalt = fcc with full occupancy of octahedral sites)
    unit = cg.get_unit_cell('rocksalt primitive', a0)
    crystal = unit.create_supercell(nsc[0], nsc[1], nsc[2])
    
    return crystal

In [10]:
crystal = create_cell_with_1H(a0, nsc=[4, 4, 4])
print(crystal.natoms)

65


In [11]:
# strain array
strain_array = np.linspace(-1.e-2, 1.e-2, 5)

for strain in strain_array:
    a0_def = (1. + strain)*a0
    crystal.scale = a0_def

    # construct subdirectory name
    a0_string = 'a0_{:6.5f}'.format(a0_def)
    subdir_name = a0_string
    jobname = 'a0 = {:6.5f}'.format(a0_def)

    vaspjob = vj.VASP_job(crystal=crystal, 
                          root_dir=root_dir, 
                          subdir_name= subdir_name, 
                          jobname=jobname,
                          path_to_INCAR=path_to_INCAR,
                          path_to_KPOINTS=path_to_KPOINTS, 
                          path_to_POTCAR=path_to_POTCAR,
                          path_to_support_dir=path_to_support_dir,
                          support_file_list=support_file_list)
    vaspjob.generate_input_deck()

for filename in global_file_list:
    path_source = os.path.join(path_to_support_global, filename)
    path_des = os.path.join(root_dir, filename)
    copyfile(path_source, path_des)
    os.chmod(path_des, 0777)